In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"F:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 10450)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|████████████████████████████████████████████████████████████████████████████| 10450/10450 [02:28<00:00, 70.60it/s]


Total cost = 1632.9621234808565

Cluster 37 assigned to Class 8. Cost: 4.165240588891101
Cluster 106 assigned to Class 28. Cost: 2.825098526927065
Cluster 268 assigned to Class 89. Cost: 1.4789972705874306
Cluster 99 assigned to Class 90. Cost: 3.745716414304838
Cluster 61 assigned to Class 91. Cost: 2.724371866976612
Cluster 114 assigned to Class 92. Cost: 2.882399779996145
Cluster 138 assigned to Class 95. Cost: 6.47692879378633
Cluster 148 assigned to Class 103. Cost: 6.163769210716216
Cluster 16 assigned to Class 111. Cost: 1.7293975354846536
Cluster 66 assigned to Class 161. Cost: 2.1046523285583794
Cluster 254 assigned to Class 162. Cost: 1.6798334189829645
Cluster 298 assigned to Class 246. Cost: 1.9149829588730702
Cluster 260 assigned to Class 250. Cost: 1.7195052173314251
Cluster 262 assigned to Class 260. Cost: 3.1560682324306084
Cluster 347 assigned to Class 275. Cost: 4.472031028050027
Cluster 194 assigned to Class 280. Cost: 3.756311213374222
Cluster 364 assigned to Class 

Cluster 342 assigned to Class 4434. Cost: 1.898429408824185
Cluster 233 assigned to Class 4450. Cost: 7.706803766656412
Cluster 318 assigned to Class 4461. Cost: 2.3620831633317074
Cluster 328 assigned to Class 4483. Cost: 2.0051839398379014
Cluster 108 assigned to Class 4497. Cost: 14.498742349567811
Cluster 291 assigned to Class 4554. Cost: 7.997094500679066
Cluster 275 assigned to Class 4563. Cost: 2.0799180402232884
Cluster 394 assigned to Class 4593. Cost: 1.5245968669767296
Cluster 361 assigned to Class 4902. Cost: 6.668952838915277
Cluster 36 assigned to Class 4904. Cost: 3.095335707984433
Cluster 199 assigned to Class 4977. Cost: 2.11861885981426
Cluster 111 assigned to Class 4998. Cost: 3.368797532066277
Cluster 20 assigned to Class 5008. Cost: 2.215751116290575
Cluster 346 assigned to Class 5039. Cost: 6.34995278516899
Cluster 160 assigned to Class 5040. Cost: 5.45100398768245
Cluster 139 assigned to Class 5042. Cost: 5.945145244090974
Cluster 310 assigned to Class 5053. Cost

Cluster 276 assigned to Class 8225. Cost: 3.379331484145738
Cluster 82 assigned to Class 8497. Cost: 10.979224223845113
Cluster 9 assigned to Class 8554. Cost: 5.107037037967453
Cluster 164 assigned to Class 8585. Cost: 2.017081359322925
Cluster 341 assigned to Class 8902. Cost: 4.1148358267036
Cluster 259 assigned to Class 8904. Cost: 5.243331153508657
Cluster 193 assigned to Class 8975. Cost: 1.9404463765457707
Cluster 31 assigned to Class 8998. Cost: 2.1155111942115474
Cluster 30 assigned to Class 9011. Cost: 3.659629937975271
Cluster 232 assigned to Class 9012. Cost: 4.194232530393801
Cluster 330 assigned to Class 9028. Cost: 3.440735748248901
Cluster 28 assigned to Class 9040. Cost: 3.259690346118325
Cluster 200 assigned to Class 9056. Cost: 3.032579938012605
Cluster 312 assigned to Class 9059. Cost: 3.9487062032088756
Cluster 3 assigned to Class 9067. Cost: 2.092125029399601
Cluster 47 assigned to Class 9073. Cost: 1.5789269930084922
Cluster 175 assigned to Class 9075. Cost: 1.69

In [5]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
